In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np

In [8]:
pwd

'c:\\Users\\jadha\\Documents\\Topic-Aware-Text-Summarization\\text_summarization\\LSA'

In [34]:
import pandas as pd
df=pd.read_csv('../../results/LDA/Lda_topics.csv')
df['Question'] = df['Question'].astype(str).apply(lambda x: x.lower())

In [36]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [37]:
df.head()

,Question,doclen,TopicId,Topic
0,"like everyone else (here in u.s), i work with ...",125,3,Relationships
1,hello dear's people. i have a fictief research...,80,3,Relationships
2,a lady buys goods worth 200 bucks from a shop ...,77,3,Relationships
3,i am turning 25 in about a month and am curren...,73,5,Exam and university
4,i'm a 34 years old married to a woman. i had a...,73,3,Relationships


In [46]:
grouped = df.groupby('TopicId')

# Retrieve a DataFrame for a specific topic, e.g., topic 0
topic_0_df = grouped.get_group(0).drop(columns='TopicId')

In [47]:
topic_0_df.head()

,Question,doclen,Topic
20,what is the cost of a lte base station (enodeb...,64,Government
53,"a rabbit, a cabbage, a fox, and a farmer are a...",60,Government
82,my iphone 6 was lost/stolen 6 days ago. i enab...,57,Government
113,i m 14 and my vocal range is from e2 to a5 (le...,54,Government
119,i m 14 and my vocal range is from e2 to a5 (le...,54,Government


In [68]:
topic_id_name_map = df[['TopicId', 'Topic']].drop_duplicates().set_index('TopicId')['Topic'].to_dict()
print(topic_id_name_map)

{3: 'Relationships', 5: 'Exam and university', 1: 'Online Business', 0: 'Government', 2: 'Social Harassment', 4: 'carrer'}


In [69]:
for topic_id, topic_name in topic_id_name_map.items():
    topic_df = grouped.get_group(topic_id).drop(columns='TopicId')
    print(f'Topic: {topic_name}')
    print(topic_df.head())
    print('\n\n')

Topic: Relationships
                                            Question  doclen          Topic
0  like everyone else (here in u.s), i work with ...     125  Relationships
1  hello dear's people. i have a fictief research...      80  Relationships
2  a lady buys goods worth 200 bucks from a shop ...      77  Relationships
4  i'm a 34 years old married to a woman. i had a...      73  Relationships
6  i have been using my girlfriend for a sexual r...      71  Relationships



Topic: Exam and university
                                             Question  doclen  \
3   i am turning 25 in about a month and am curren...      73   
7   you have 100 coins laying flat on a table, eac...      68   
16  i belong to obc. my ctc is 7 lacs per annum bu...      64   
17  i have 2.6 years of it experience. i got marri...      64   
30  i have scored 72% marks in 10th, 80% marks in ...      62   

                  Topic  
3   Exam and university  
7   Exam and university  
16  Exam and university 

In [92]:
with open('topic_summary.txt', 'w',encoding="utf-8") as f:
    f.write('Topic Summary\n\n')
    for key,value in topic_id_name_map.items():
        f.write(f'Topic: {value}\n')
        topic_df = grouped.get_group(key).drop(columns='TopicId')
        print(f'Topic: {value}',topic_df.head())
        f.write(f'Total number of questions: {topic_df.shape[0]}\n')
        documents = topic_df['Question'].values
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(documents)
        n_components = 10  # Choose number of concepts to retain
        lsa = TruncatedSVD(n_components=n_components)
        lsa_matrix = lsa.fit_transform(tfidf_matrix)
        sentence_scores = np.sum(lsa_matrix, axis=1)
        top_sentence_indices = sentence_scores.argsort()[-50:]  # Select top 50 sentences for summary
        summary = [documents[i] for i in sorted(top_sentence_indices)]
        print("Summary:", " ".join(summary))
        f.write("Summary: " + "\n ".join(summary) + '\n\n')



Topic: Relationships                                             Question  doclen          Topic
0  like everyone else (here in u.s), i work with ...     125  Relationships
1  hello dear's people. i have a fictief research...      80  Relationships
2  a lady buys goods worth 200 bucks from a shop ...      77  Relationships
4  i'm a 34 years old married to a woman. i had a...      73  Relationships
6  i have been using my girlfriend for a sexual r...      71  Relationships
Summary: what are good gifts for a foreign visitor to bring when they're invited to someone's home in iraq for the first time? what are good gifts for a foreign visitor to bring when they're invited to someone's home in turkey for the first time? what are good gifts for a foreign visitor to bring when they're invited to someone's home in turkey for the first time? what are good gifts for a foreign visitor to bring when they're invited to someone's home in singapore for the first time? what are good gifts for a foreign